# Edit Your Expectation Suite
Use this notebook to recreate and modify your expectation suite:

**Expectation Suite Name**: `adubinsky.ods_traffic.warning`

We'd love it if you **reach out to us on** the [**Great Expectations Slack Channel**](https://greatexpectations.io/slack)

In [ ]:
import datetime
import great_expectations as ge
import great_expectations.jupyter_ux
from great_expectations.checkpoint import LegacyCheckpoint
from great_expectations.data_context.types.resource_identifiers import ValidationResultIdentifier

context = ge.data_context.DataContext()

# Feel free to change the name of your suite here. Renaming this will not
# remove the other one.
expectation_suite_name = "adubinsky.ods_traffic.warning"
suite = context.get_expectation_suite(expectation_suite_name)
suite.expectations = []

batch_kwargs = {'data_asset_name': 'adubinsky.ods_traffic', 'datasource': 'gp', 'limit': 1000, 'schema': 'adubinsky', 'table': 'ods_traffic'}
batch = context.get_batch(batch_kwargs, suite)
batch.head()

## Create & Edit Expectations

Add expectations by calling specific expectation methods on the `batch` object. They all begin with `.expect_` which makes autocompleting easy using tab.

You can see all the available expectations in the **[expectation glossary](https://docs.greatexpectations.io/en/latest/reference/glossary_of_expectations.html?utm_source=notebook&utm_medium=create_expectations)**.

### Table Expectation(s)

In [ ]:
batch.expect_table_column_count_to_equal(value=7)

In [ ]:
batch.expect_table_columns_to_match_ordered_list(column_list=['user_id', 'timestamp', 'device_id', 'device_ip_addr', 'bytes_sent', 'bytes_received', 'src_name'])

### Column Expectation(s)

#### `user_id`

In [ ]:
batch.expect_column_kl_divergence_to_be_less_than(column='user_id', threshold=0.01)

#### `device_id`

In [ ]:
batch.expect_column_values_to_match_regex(column='device_id', regex='^(d)[\d]{3}')

#### `device_ip_addr`

In [ ]:
batch.expect_column_values_to_match_regex(column='device_ip_addr', regex='((?:(?:25[0-5]|2[0-4]\d|((1\d{2})|([1-9]?\d)))\.){3}(?:25[0-5]|2[0-4]\d|((1\d{2})|([1-9]?\d))))')

#### `bytes_sent`

In [ ]:
batch.expect_column_min_to_be_between(column='bytes_sent', max_value=10000, min_value=0)

#### `bytes_received`

In [ ]:
batch.expect_column_max_to_be_between(column='bytes_received', max_value=50000, min_value=0)

#### `src_name`

In [ ]:
batch.expect_column_distinct_values_to_be_in_set(column='src_name', value_set=['stg_traffic'])

## Save & Review Your Expectations

Let's save the expectation suite as a JSON file in the `great_expectations/expectations` directory of your project.
If you decide not to save some expectations that you created, use [remove_expectation method](https://docs.greatexpectations.io/en/latest/autoapi/great_expectations/data_asset/index.html?highlight=remove_expectation&utm_source=notebook&utm_medium=edit_expectations#great_expectations.data_asset.DataAsset.remove_expectation).

Let's now rebuild your Data Docs, which helps you communicate about your data with both machines and humans.

In [ ]:
batch.save_expectation_suite(discard_failed_expectations=False)

results = LegacyCheckpoint(
    name="_temp_checkpoint",
    data_context=context,
    batches=[
        {
          "batch_kwargs": batch_kwargs,
          "expectation_suite_names": [expectation_suite_name]
        }
    ]
).run()
validation_result_identifier = results.list_validation_result_identifiers()[0]
context.build_data_docs()
context.open_data_docs(validation_result_identifier)